# Amazon Review Clustering

Miloy Ajmera (majmera@andrew.cmu.edu) / 17-634: Applied Machine Learning, Spring 2023

In [ ]:
import os
import sklearn
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
import random

import seaborn as sns  # We'll mostly use seaborn, a stats visualization library
import matplotlib.pyplot as plt  # Seaborn wrap Matplotlib for visualization. We'll occasionaly want to make custom changes to the plots.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/Othercomputers/My PC/Carnegie Mellon University Notes/Semester 3 - Spring 2023/Applied Machine Learning/Homework Assignments/Amazon/"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 amazon
'Amazon Review Clustering.ipynb'
'Amazon Review Clustering - MetaData.ipynb'
'Amazon Review Clustering - Small Sample.ipynb'
 amazon.tar.gz


In [ ]:
filepath = "/content/drive/Othercomputers/My PC/Carnegie Mellon University Notes/Semester 3 - Spring 2023/Applied Machine Learning/Homework Assignments/Amazon/"

## Load the Amazon Review dataset.
Read and prepare the data. Consider how you plan to deal with missing data: will you remove columns, if so, under what conditions? Will you attempt to impute any data?

In [ ]:
# get the contents and size of the json file - 2 mins 0 secs
filename = 'amazon/Home_and_Kitchen.json'
# Load the dataframes
num_lines = sum(1 for line in open(filepath + filename))
print(num_lines)

21928568


In [ ]:
# Using random library and examples from https://docs.python.org/3/library/random.html
# Taking random samples - 0 secs
lines_to_be_considered = num_lines / 100000
chunk_size = int(num_lines / 1000)
total_range = range(0, num_lines)

ranges = []
for i in range(0, len(total_range), chunk_size) :
  ranges.append(total_range[i:i+chunk_size])

random_values = []
for split_range in ranges:
  random_values.append(random.sample(split_range, int(len(split_range)/100)))

random_list = [random_value for sublist in random_values for random_value in sublist]

sample_list = random.sample(random_list, int(len(random_list)/10))
set_samples = set(sample_list)

print(len(set_samples))

21900


In [ ]:
# get ranges - 0 secs
df_range = range(0, num_lines)
ranges = []
for i in range(0, len(df_range), int(len(df_range)/10)) :
  ranges.append(df_range[i:i+int(len(df_range)/10)])

ranges = ranges[:-1]
print(ranges)


[range(0, 2192856), range(2192856, 4385712), range(4385712, 6578568), range(6578568, 8771424), range(8771424, 10964280), range(10964280, 13157136), range(13157136, 15349992), range(15349992, 17542848), range(17542848, 19735704), range(19735704, 21928560)]


In [ ]:
# understand data - 10 mins
import csv
import json
with open(filepath + 'amazon/samplefile.json', 'w') as writer_file:
  json_file = open(filepath + filename, "r")
  data = []
  for i, line in enumerate(json_file):
      if i in set_samples:
        data = json.loads(line)
        json.dump(data, writer_file)
        writer_file.write('\n')


In [ ]:
# extract data into a dataframe
json_file = open(filepath + 'amazon/samplefile.json', "r")
data = []
for i, line in enumerate(json_file):
      data.append(json.loads(line))
df = pd.DataFrame.from_records(data)
df.head()


,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image,vote
0,5.0,True,"02 6, 2013",A2VASVQ9IRBWAO,0983124248,{'Format:': ' Spiral-bound'},Amazon Customer,"It was more than I expected, It had a cover th...",My Sticker Book Princess swing,1360108800,NaN,NaN
1,5.0,True,"08 4, 2016",A1OB19ILFMIABU,1579280633,NaN,Amazon Customer,A very nice piece for my collection. Very Sat...,Thank you.,1470268800,NaN,NaN
2,5.0,True,"07 28, 2016",A25RF1N6EBP7IE,9170011451,{'Material Type:': ' 48 curtain clips'},Allen,I ended up taking the two pieces apart to use ...,Great,1469664000,[https://images-na.ssl-images-amazon.com/image...,5
3,4.0,True,"04 10, 2016",A29SCE5YIHH3YE,9170011451,{'Material Type:': ' Stainless Steel'},Ahkeith Salley,Its good. wire is a tad thinner than expected...,Not bad.,1460246400,NaN,NaN
4,1.0,True,"07 16, 2015",AQE429IJXNSWS,B00000JGRP,{'Color:': ' Black Chrome'},JT,I should have known better. I had hoped to use...,I should have known better. I had hoped to use...,1437004800,NaN,2


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21900 entries, 0 to 21899
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   overall         21900 non-null  float64
 1   verified        21900 non-null  bool   
 2   reviewTime      21900 non-null  object 
 3   reviewerID      21900 non-null  object 
 4   asin            21900 non-null  object 
 5   style           12148 non-null  object 
 6   reviewerName    21899 non-null  object 
 7   reviewText      21886 non-null  object 
 8   summary         21892 non-null  object 
 9   unixReviewTime  21900 non-null  int64  
 10  image           642 non-null    object 
 11  vote            2890 non-null   object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 1.9+ MB


In [ ]:
# extract the dataframe into a new dataframe
df_review = df[['reviewText']]
df_review

,reviewText
0,"It was more than I expected, It had a cover th..."
1,A very nice piece for my collection. Very Sat...
2,I ended up taking the two pieces apart to use ...
3,Its good. wire is a tad thinner than expected...
4,I should have known better. I had hoped to use...
...,...
21895,This is definitely NOT worth the money and the...
21896,They're cute. Lightweight. The gray almost has...
21897,Ordered pillows for these and they fit as inst...
21898,Received the toaster yesterday afternoon. We ...


### Now perform LDA on the extracted dataset

In [ ]:
from gensim import models, similarities
from gensim.corpora.dictionary import Dictionary
# Plus a few other assorted inputs.
import numpy as np
from sklearn.datasets import fetch_20newsgroups

In [ ]:
# Clean the data
df_review['reviewText'] = df_review['reviewText'].apply(lambda x: np.nan if len(str(x)) < 2 else x)
df_review = df_review.dropna()
df_review['reviewText'] = df_review['reviewText'].apply(lambda x: x.lower())
df_review.head()

In [ ]:
# Removing stop words
import pandas as pd
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
df_review['reviewText'] = df_review['reviewText'].apply(lambda x: ' '.join([word for word in str(x).split() if str(word) not in (stop_words)]))
df_review

In [ ]:
# Here we start from full sentences again to help the NLP library understand the types of words. 
# Note: this can take quite a while! - 5 mins
lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    return [lemmatizer.lemmatize(word) for word in word_tokenize(text)]
df_review['reviewText'] = df_review['reviewText'].apply(lemmatize_text)
df_review.head()

In [ ]:
# And then stem all words.
from gensim.parsing.porter import PorterStemmer

stemmer = PorterStemmer()

def stem_text(word):
    return stemmer.stem(word)

df_review['reviewText'] = df_review['reviewText'].apply(lambda x: [stemmer.stem(word) for word in x])
df_review

In [ ]:
# Create a corpus from a list of texts.
# The dictionary just extracts and numbers each distinct word.
dictionary = Dictionary(df_review['reviewText'])
# A corpus is a sparse datastore containing the number of times each word appears in each document.
corpus = [dictionary.doc2bow(text) for text in df_review['reviewText']]


In [ ]:
# Print a sample of dictionary items.
top_words_in_doc_0 = sorted(corpus[0], key=lambda e: e[1], reverse=True)[:10]
for word_index, count in top_words_in_doc_0:
  print(f'{dictionary[word_index]}   \t\t index: {word_index}\tcount: {count:,}')

## Models Trained on Text

In [ ]:
# Let's start with a simple TF-IDF model. Worth noting: SKLearn also has a Tfidf vectorizer.
model = models.TfidfModel(corpus)

In [ ]:
# What are the most _salient_ words in document 0?
top_tfidf_words = sorted(model[corpus[0]], key=lambda e: e[1], reverse=True)[:10]
for word_index, score in top_tfidf_words:
  print(f'{dictionary[word_index]}   \t\t index: {word_index} \t\t tf-idf: {score:.2f}')

# What's different from the previous ranking? What do you think the topic label might be?

move   		 index: 3 		 tf-idf: 0.65
book   		 index: 0 		 tf-idf: 0.44
surpris   		 index: 4 		 tf-idf: 0.37
cover   		 index: 1 		 tf-idf: 0.31
thi   		 index: 5 		 tf-idf: 0.29
expect   		 index: 2 		 tf-idf: 0.24


In [ ]:
# Okay, let's do something more exciting: build an LDA model.
# Note: we could also do this with SKLearn using LinearDiscriminantAnalysis.

# Let's start with 20 topics.
num_topics = 20
model = models.LdaMulticore(corpus, num_topics=num_topics, id2word=dictionary, passes=10, workers=4, dtype=np.float64)

In [ ]:
# Let's again look at the most salient words per topic, but note that we no longer
# have "labels" now.
for ix in range(num_topics):
  top10 = np.argsort(model.get_topics()[ix])[-10:]
  print(f'{ix}:  {" ".join([dictionary[index] for index in top10])}')  # See any patterns?

0:  look bake size clean cook easi great perfect us pan
1:  price sturdi mattress assembl great look bed togeth easi put
2:  open return broken box replac work us chair bottl on
3:  soap get amazon bought knive time i us knife on
4:  ok set on can t small towel like us look
5:  bag made look well product nice price great qualiti good
6:  get on it glass would stick like shower curtain us
7:  nice rice take i lid put contain space top us
8:  like cover mattress size fit bed comfort soft wash sheet
9:  order like perfect nice exactli expect beauti pictur color look
10:  well make recommend cut easi work ic us product great
11:  like get bui i us s review would pillow on
12:  wast time would made great well us work worth monei
13:  block type hook b i video class link imag a
14:  us want product order get awesom it smell love candl
15:  on tea maker i water pot make us cup coffe
16:  time like get keep water vacuum on ve us i
17:  friend christma cute them daughter bought great gift it lo

In [ ]:
# How coherent are these topics?
from gensim.models.coherencemodel import CoherenceModel

cm = CoherenceModel(model=model, texts=df_review['reviewText'], dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
coherence

-2.5940560507037076

In [ ]:
# How coherent are these topics?
from gensim.models.coherencemodel import CoherenceModel

cm = CoherenceModel(model=model, texts=df_review['reviewText'], dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
coherence

0.48029353895121607

In [ ]:
with open(filepath + r'amazon.model', 'wb') as f:
    pickle.dump(model, f)